# Init OpenSky into Kafka

In [2]:
import requests
from kafka import KafkaProducer
import json

def send_opensky_to_kafka(topic, url, fields={}):

    # Kafka configuration
    kafka_config = {
        'bootstrap_servers': 'kafka1:9092',  # Update with your Kafka broker
    }

    # Initialize Kafka Producer
    producer = KafkaProducer(
        bootstrap_servers=kafka_config['bootstrap_servers'],
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )

    # Fetch data from ThingSpeak
    
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        states = data.get("states", [])

        # Publish each entry to Kafka
        for state in states:
            for field in fields:
                state[fields[field]] = state.pop(field)
            producer.send(topic, value=state)
            #print(f"Sent: {feed}")

        # Ensure all messages are sent
        producer.flush()
        print(f"Sent {len(states)} records.")
    else:
        print(f"Failed to fetch data: {response.status_code}")

## Initiate beginning

In [ ]:
from datetime import datetime, timedelta

# https://thingspeak.mathworks.com/channels/1785844 : Wind Power Smart Monitor
fields = {
    "on_ground": "on_ground",
    "latitude": "latitude",
    "longitude": "longitude"
}

api_url = "https://opensky-network.org/api/states/all"

# date initiale
initial_date_str = "2025-01-25 16:00:00"
# durée de récupération (minutes)
step = 60

date_format = "%Y-%m-%d %H:%M:%S"
initial_date = datetime.strptime(initial_date_str, date_format)
end_date = initial_date + timedelta(minutes=step)

start = initial_date.strftime(date_format)
end = end_date.strftime(date_format)

## Exemple with 10 calls.

In [10]:
import time 
for i in range(10):
    print(f"{start}-{end}")        
    send_opensky_to_kafka("wind", api_url, fields)
    time.sleep(5)
    # initial_date = end_date
    # end_date = initial_date + timedelta(minutes=step)
    # start = initial_date.strftime(date_format)
    # end = end_date.strftime(date_format)
